## 1.词法分析

| 类型  | 编号 | 类型 | 编号 | 类型 | 编号 |  类型  | 编号 |
| :---: | :--: | :--: | :--: | :--: | :--: | :----: | ---- |
| begin |  1   |  +   |  13  |  <=  |  22  | STRING | 30   |
|  if   |  2   |  -   |  14  |  >   |  23  |   .    | 31   |
| then  |  3   |  *   |  15  |  >=  |  24  |   ，   | 32   |
| while |  4   |  /   |  16  |  =   |  25  |   #    | 33   |
|  do   |  5   |  :   |  17  |  ;   |  26  |        |      |
|  end  |  6   |  :=  |  18  |  (   |  27  |        |      |
|  VAR  |  10  |  <   |  20  |  )   |  28  |        |      |
|  NUM  |  11  |  <>  |  21  | CHAR |  29  |        |      |
|       |      |      |      |      |      |        |      |



In [4]:
import unicodedata
# 判断字符串是否是数值,可以判断浮点数(3.14)和科学计数法(1e-3)和不带符号的数值
def is_number(s):
    try:         # 看是否可以转换成浮点数
        float(s)
        return True
    except ValueError:    # 如果出现异常则说明无法转换
        pass
  
    try:      # 看是否可以用unicodedata转换成数字
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):   # 如果出现异常则说明无法转换
        pass
 
    return False

In [5]:
def is_var(token):   # 判断它是否为变量，判断char(char|digit)*  类型的字符串
    if not token[0].is_alpha():   
        return False
    for c in token:
        if c.isalpha or c.isdigit:
            next
        else:
            return False
    return True

In [9]:
keyword2num = {     # 将标识符转换成编号
    '+': 13,
    '-': 14,
    '*': 15,
    '/': 16,
    '=': 25,
    '(': 27,
    ')': 28,
    '#': 0 ,
    ';': 26,
    "'": 27,
    '"': 28,
    '.': 31,
    ',': 32
}
token2num = {    # 将类型名转换成编号
    'begin': 1, 'if': 2, 'then': 3, 'while': 4, 'do': 5, 'end': 6,'VAR': 10,'NUM': 11,'+': 13,'-':14,'*':15,'/':16,
    ':':17, ':=':18,'<':20,'<>':21,'<=':22,'>':23,'>=':24,'=':25,';':26,'(':27,')':28,'#':0,'CHAR':29,'STRING':30,'.':31,',':32
}
def is_keyword(s):# 判断是否是标识符
    if s=='+' or s == '-' or s == '*' or s == '/' or s == '=' or s == '(' or s== ')' or s=='#' or s== ';'  or s == ',':
        return True
    return False

In [16]:
def lex_parse(s):
    lineno = 0
    lex_list = []
    counter = 0;
    while counter < len(s):
        if s[counter] == ' ':   # 如果是空格则跳过这个字符
            counter+=1
            continue
        if s[counter] == '\n':  # 如果回车则行数加一
            lineno += 1
            continue
        if s[counter].isdigit():  # 如果是数字，则判断是否是数值
            start = counter
            counter += 1
            if counter < len(s):
                while s[counter] != ' ' and not is_keyword(s[counter]):
                    counter += 1
                    if counter >= len(s):
                        break
            digit_end = counter
            if is_number(s[start:digit_end]):   
                lex_list.append([token2num["NUM"],s[start:digit_end],lineno])
                continue
            else:
                print('error in {line},becouse {token} is not a right number value!'.format(line=lineno,token=s[start:digit_end]))
                break
        if s[counter].isalpha():  # 如果是字母，则判断是变量还是标识符字符串
            start = counter
            counter += 1
            if counter < len(s):
                while s[counter].isalpha() or s[counter].isdigit():
                    counter += 1
                    if counter >= len(s):
                        break
            var_end = counter-1
            var_str = s[start:var_end+1]
            if var_str == 'begin':
                lex_list.append([token2num["begin"], 'begin',lineno])
            elif var_str == 'if':
                lex_list.append([token2num['if'], 'if',lineno])
            elif var_str == 'then':
                lex_list.append([token2num['then'], 'then',lineno])
            elif var_str == 'while':
                lex_list.append([token2num['while'], 'while',lineno])
            elif var_str == 'do':
                lex_lsit.append([token2num['do'], 'do',lineno])
            elif var_str == 'end':
                lex_list.append([token2num['end'], 'end',lineno])
            else:
                lex_list.append([token2num["VAR"],var_str,lineno])
            continue
        if is_keyword(s[counter]):       # 是否是标识符字符，逗号、分号、等于
            lex_list.append([keyword2num[s[counter]],s[counter],lineno])
            counter += 1
            continue
        if s[counter] == ':':         # 对一些特殊的标识符特殊处理，：/  ：=
            counter+=1
            if counter < len(s):
                if s[counter] == '=':
                    lex_list.append([token2num["="],':=',lineno])
                    counter += 1
                else:
                    lex_list.append([token2num[":"],':',lineno])
            continue
        if s[counter] == '<':    # 区分  < / <= / <>
            counter += 1
            if counter < len(s):
                if s[counter] == '=':
                    lex_list.append([token2num['<='],'<=',lineno])
                    counter+=1
                elif s[counter] == '>':
                    lex_list.append([token2num['<>'],'<>',lineno])
                    counter += 1
                else:
                    lex_list.append([token2num['<'],'<',lineno])
            continue
        if s[counter] == '>':
            counter += 1
            if counter < len(s):
                if s[counter] == '=':
                    lex_list.append([token2num['>='],'>=',lineno])
                    counter += 1
                else:
                    lex_list.append([token2num['>'],'>',lineno])
            continue
        if s[counter] == '\"':     # 判断是否是字符串
            counter += 1
            start = counter
            if counter < len(s):
                while s[counter] != '\"':      # 判断是否是不是少一个引号匹配
                    if counter+1 < len(s):
                        counter+=1
                    else:
                        print("error in {line},can\'t match another ”".format(line=lineno))
                        break
            str_end = counter
            lex_list.append([token2num['STRING'],s[start:str_end],lineno])
            counter += 1
            continue
        if s[counter] == '\'':               # 判断是否是字符
            counter += 1
            lex_list.append([token2num['CHAR'],s[counter],lineno])
            counter+=1
            if s[counter] == '\'':   # 判断是否有 '和它匹配
                counter += 1
                continue
            else:
                print("error in {line},can\'t match another ’".format(line=lineno))
                break
    return lex_list

In [17]:
lex_list = lex_parse("String test = \"abcd")

error in 0,can't match another ”


### 生成的token表

In [13]:
import pandas as pd
token_table = pd.DataFrame(lex_list,columns = ['编号','类型值','行号'])
token_table

,编号,类型值,行号
0,10,const,0
1,10,count,0
2,25,=,0
3,11,10,0
4,32,",",0
5,10,sum,0
6,25,=,0
7,11,81.5,0
8,32,",",0
9,10,char1,0


## 2.语法分析

In [250]:
def seg_parse(s):
    global lex_res
    global count
    count = 0
    lex_res+=lex_parse(s)
    while(True):
        if count < len(lex_res)and lex_res[count][0]!=0:
            express()
            match(token2num[';'])      # list-> expr;list
        else:
            break

In [251]:
def express():
    global lex_res
    global count
    term()    # express->term
    while(True):
        if lex_res[count][0] == token2num['+'] or lex_res[count][0] == token2num['-']:   # expr->expr+term {print('+')} 
            t = lex_res[count][1]                                                        #or expr->expr-term {print('-')}
            match(lex_res[count][0])
            term()
            print(t,end='')
        else:
            break

In [252]:
def term():
    global lex_res
    global count 
    factor()      # term -> factor
    while True:
        if lex_res[count][0] == token2num['*'] or lex_res[count][0] == token2num['/']:   # term->term*factor {print('*')}
            t = lex_res[count][1]                                                        # term->term/factor  {print('/')}
            match(lex_res[count][0])
            factor()
            print(t,end='')
        else:
            break

In [253]:
def factor():
    global lex_res
    global count
    if lex_res[count][0] == token2num['(']:         # factor -> ( express )
        match(token2num['('])
        express()
        match(token2num[')'])
    elif lex_res[count][0] == token2num['NUM']:    # factor -> NUM  {print(NUM.val)}
        print(lex_res[count][1],end='')
        match(token2num['NUM'])
    elif lex_res[count][0] == token2num['VAR']:     # factor -> VAR {print(factor.val)}
        print(lex_res[count][1],end='')
        match(token2num['VAR'])
    else:
        print('syntx error in factor')

In [254]:
def match(c):
    global lex_res
    global count
    if lex_res[count][0] == c:
        count += 1
    else:
        print('synax error in match, error in {line} line, in matching {word};'.format(line=lex_res[count][2],word=lex_res[count][1]))

In [255]:
lex_res = []
count = 0
seg_parse('1 + 2-3+1+3*(4+5);1+2+3+4;')

12+3-1+345+*+12+3+4+

## 符号表查看

In [258]:
import pandas as pd
symbol_table = pd.DataFrame(lex_res,columns=['编号','符号值','所在行'])
symbol_table

,编号,符号值,所在行
0,11,1,0
1,13,+,0
2,11,2,0
3,14,-,0
4,11,3,0
5,13,+,0
6,11,1,0
7,13,+,0
8,11,3,0
9,15,*,0


## 3.说明语句的词法分析器


In [388]:
state_lex = []
state_list = []
counter = 0
def state_syntax_parse(s): 
    global state_lex
    global state_list
    global counter
    counter = 0
    state_lex = lex_parse(s)
#     print(state_lex)
    # 首先判断是否是说明语句
    if(state_lex[counter][1] != 'const'):
        print("It is not a constant declaration statement!")
        print("Please input a string again!")
        return
       # 如果是说明语句，跳过第一个语句
    while counter < len(state_lex):    # 对const后的语句进行分析
        counter += 1
        if state_lex[counter][0] != token2num['VAR']:    # 开头必须是个变量名，否则会报错
            print("error in "+str(state_lex[counter][2])+",because "+str(state_lex[counter][1])+"is not a VAR name")
            return
        name = state_lex[counter][1]
        counter += 1
        if state_lex[counter][0] != token2num['=']:    # 变量名后必须要跟等号
            print("error in "+str(state_lex[counter][2])+",because the state has a correct struct")
            return 
        counter += 1
        if state_lex[counter][0] == token2num['NUM']:  # 等号后可以跟这几种类型的值，通过lex_parse获取这几种
            state_list.append([name,'NUM',state_lex[counter][1]])
        elif state_lex[counter][0] == token2num['STRING']:
            state_list.append([name,'STRING',state_lex[counter][1]])
        elif state_lex[counter][0] == token2num['CHAR']:
            state_list.append([name,'CHAR',state_lex[counter][1]])
        else:
            print("error in "+str(state_lex[counter][2])+",because the "+str(state_lex[counter][1])+" is a correct type")
            return
        counter += 1
        if counter < len(state_lex):
            if state_lex[counter][0] == token2num[';']:
                break
            if state_lex[counter][0] != token2num[',']:
                print("error in "+str(state_lex[counter][2])+",because the state has a correct struct")
                counter+=1
    return 

In [389]:
state = "const count = 10,sum=81.5, char1='f',max=169,str2=\"54*sdaw\";"
state_syntax_parse(state)

## 变量表

In [387]:
var_symbol_table = pd.DataFrame(state_list,columns=['变量名','变量类型','变量值'])
var_symbol_table

,变量名,变量类型,变量值
0,count,NUM,10
1,sum,NUM,81.5
2,char1,CHAR,f
3,max,NUM,169
4,str2,STRING,54*sdaw


## 4.基于预测分析方法的表达式语法分析器

### 预测分析表
|      |    m    |    *    |    +    |    (    |    )    |    #    |
| :--: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
|  S   |  S->AT  | *error* | *error* |  S->AT  | *error* | *error* |
|  A   |  A->BU  | *error* | *error* |  A->BU  | *error* | *error* |
|  T   | *error* | *error* | T->+AT  | *error* |  T->ε   |  T->ε   |
|  U   | *error* | U->*BU  |  U->ε   | *error* |  U->ε   |  U->ε   |
|  B   |  B->m   | *error* | *error* | B->(S)  | *error* | *error* |





In [390]:
# 构建存储分析表的数据结构, 构造可以逆向入栈的list
table = {}
table['S'] = {'m':['T','A'] ,'*':['error']     ,'+':['error']     ,'(':['T','A'],')':['error']     ,'#':['error']}
table['A'] = {'m':['U','B'] ,'*':['error']     ,'+':['error']     ,'(':['U','B'],')':['error']     ,'#':['error']}
table['T'] = {'m':['error'] ,'*':['error']     ,'+':['T','A','+'] ,'(':['error'],')':['$']     ,'#':['$']}
table['U'] = {'m':['error'] ,'*':['U','B','*'] ,'+':['$']         ,'(':['error'],')':['$']         ,'#':['$']}
table['B'] = {'m':['m']     ,'*':['error']     ,'+':['error']     ,'(':['error'],')':[')','S','('] ,'#':['error']}
final_token = ['m','*','+','(',')']
temp_token = ['S','A','T','U','B']
error_status = ['error','error1','error2','error3']

In [391]:
def error_check(error_type,stack):
    if error_type == 'error':
        print('ERROR in matching temp_token '+stack[-1])

In [354]:
printContent = []
def printing(stack, s_temp,content):    # 打印预测分析法的分析过程
    printContent.append([''.join(stack),''.join(s_temp),content])

In [359]:
def LL1(s):
    global printContent
    printContent = []
    # 用python的list实现栈结构
    stack = []
    # 初始栈
    stack.append('#')
    stack.append('S')
    s_temp = list(s)
    counter = 0
    token = s[counter]
    while True:
        if stack[-1] == '#' and token == '#':   # 如果stack栈底和token都到底，即分析成功
            print('Success')
            break
        if stack[-1] in final_token:   # 如果栈顶是终结符则匹配
            if stack[-1] == token:
                printing(stack,s_temp,'{token}匹配'.format(token=token))
                stack.pop()
                s_temp = s_temp[1:]
                counter+=1
                token = s[counter]
                continue
            else:                     #   匹配不成功就报错
                print('ERROR in matching token '+stack[-1])
                return 
        else:                         # 栈顶是非终结符
            top = stack[-1]
            if table[top][token][0] not in  error_status:    # 是否是正确的匹配规则
                printing(stack,s_temp,"{token}->{dist}".format(token=top,dist=''.join(table[top][token])))
                stack.pop()
                for item in table[top][token]:
                    if item == '$':
                        continue
                    else:
                        stack.append(item)
                continue
            else:
                # 匹配错误处理程序
                error_check(table[top][token][0],stack)
#                 print('ERROR in matching temp_token '+stack[-1])
                return

In [360]:
LL1('m+m*m#')

Success


### LL1推导过程

In [362]:
ll_table = pd.DataFrame(printContent,columns=['Stack','String','Rule'])
ll_table

,Stack,String,Rule
0,#S,m+m*m#,S->TA
1,#TA,m+m*m#,A->UB
2,#TUB,m+m*m#,B->m
3,#TUm,m+m*m#,m匹配
4,#TU,+m*m#,U->$
5,#T,+m*m#,T->TA+
6,#TA+,+m*m#,+匹配
7,#TA,m*m#,A->UB
8,#TUB,m*m#,B->m
9,#TUm,m*m#,m匹配
